In [2]:
import math
import tensorflow as tf
import numpy as np
import os

from tensorflow.contrib import rnn
tf.reset_default_graph()
tf.set_random_seed(777)


def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

#load data
xy = np.genfromtxt('/Users/yeseo/Desktop/taxi_data/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)

#data preprocessing
train_size = int(len(xy)*0.7)

xy= xy[:,:27]
test_set = xy[train_size:]
a = xy[:,:2]
b = xy[:,2:]
data_max = np.max(b,1)
data_min = np.min(b,1)
b= MinMaxScaler(b)
xy = np.hstack((a,b))
train_set = xy[:train_size]

#parameters
seq_length =6
data_dim =27
hidden_dim = 27
output_dim = 25
learning_rate = 0.1
iterations = 1001

def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#make train_set&test_set 
trainX, trainY = build_dataset(train_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

X = tf.placeholder(tf.float32,[None, seq_length,data_dim],name = "X")
Y = tf.placeholder(tf.float32,[None,25], name = "Y")



#make LSTM CELL
with tf.variable_scope("rnn"):
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    #multi_cell1 = tf.contrib.rnn.MultiRNNCell([cell1]*5)
    outputs,_states = tf.nn.dynamic_rnn(cell,X,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim,activation_fn = None)
    
    loss =tf.reduce_mean(tf.square(Y_pred-Y), name = "loss")
    train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

# make saver
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(iterations):
        _, step_loss = sess.run([train,loss],feed_dict={X:trainX, Y:trainY})
        if i%500 == 0:
            print("[step: {}] loss: {}".format(i,step_loss))
    
    save_path = saver.save(sess, "/Users/yeseo/ML_with_Taxidata/code/for_solution/model/my_model.ckpt")
    
    #7th value from initial data
    test_predict = sess.run(Y_pred, feed_dict = {X:[testX[0,:,:]]})
    
    #initial data shape = 6,27
    initial_data = testX[0,:,:]
    
    
    #day&time for 7th data
    if initial_data[5,1] != 47:
        day_time = np.array([[initial_data[5,0],(initial_data[5,1]+1)]])
    else:
        if initial_data[5,0] == 7:
            day_time = np.array([[1,0]])
        else:
            day_time = np.array([[(initial_data[5,0]+1),0]])
    
    time_array = day_time
    
    #make 7th data
    seventh_data = np.hstack((day_time,test_predict))
    
    #1~5 data + 7th data
    new_testX = np.vstack((initial_data[1:,:],seventh_data))
    
    #select range to predict
    for j in range(48):
        #8th value from second data
        temp = sess.run(Y_pred,feed_dict = {X:[new_testX]})
        
        test_predict = np.vstack((test_predict,temp))
        
        if new_testX[5,1] != 47:
            day_time = np.array([[new_testX[5,0],(new_testX[5,1]+1)]])
        else:
            if new_testX[5,0] == 7:
                day_time = np.array([[1,0]])
            else:
                day_time = np.array([[(new_testX[5,0]+1),0]])
        time_array = np.vstack((time_array,day_time))
        seventh_data = np.hstack((day_time,test_predict[[-1],:]))
        new_testX = np.vstack((new_testX[1:,:],seventh_data))

[step: 0] loss: 0.21923425793647766
[step: 500] loss: 0.0017684613121673465
[step: 1000] loss: 0.0015603110659867525


In [5]:
np.shape(testX)

(1204, 6, 27)

In [2]:
# make data to original
def ReturnToOriginal(data,data_max,data_min):
    data_max = data_max+(1e-7)
    for i in range(0,len(data)):
        data[i,:]= (data[i,:]*data_max[i])-(data[i,:]-1)*data_min[i]
    return data

test_predict_to_original = ReturnToOriginal(test_predict,data_max[train_size+seq_length:],data_min[train_size+seq_length:])

In [3]:
test_predict_to_original = np.array(test_predict_to_original,np.int32)

In [4]:
test_predict_to_original = np.hstack((time_array,test_predict_to_original))

In [6]:
test_predict_to_original = np.array(test_predict_to_original,np.int32)

In [9]:
test_predict_to_original = np.array(test_predict_to_original,str)

In [10]:
test_predict_to_original

array([['3', '44', '8096', ..., '9398', '10709', '10677'],
       ['3', '45', '8195', ..., '9729', '11194', '11197'],
       ['3', '46', '8337', ..., '9233', '10526', '10647'],
       ...,
       ['4', '42', '9055', ..., '10182', '11782', '11714'],
       ['4', '43', '8796', ..., '9641', '11313', '11233'],
       ['4', '44', '9837', ..., '11072', '13217', '13193']], dtype='<U11')

In [12]:
 for i in range(len(test_predict_to_original)):    
    with open("/Users/yeseo/desktop/prediction_12.txt",'a') as fa:
        for j in range(27):
            fa.write(test_predict_to_original[i,j]+",")
        fa.write('\n')
        fa.flush()

In [3]:
#make data to json format
import json
from collections import OrderedDict
file_data = OrderedDict()

day_time = 0

file_data["sigungu"] = {"Seoul_Jongno-gu":str(test_predict_to_original[:,0]),
                            "Seoul_Jung-gu":str(test_predict_to_original[:,1]),
                            "Seoul_Yongsan-gu":str(test_predict_to_original[:,2]),
                            "Seoul_Seongdong-gu":str(test_predict_to_original[:,3]),
                            "Seoul_Gwangjin-gu":str(test_predict_to_original[:,4]),
                            "Seoul_Dongdaemun-gu":str(test_predict_to_original[:,5]),
                            "Seoul_Jungnang-gu":str(test_predict_to_original[:,6]),
                            "Seoul_Seongbuk-gu":str(test_predict_to_original[:,7]),
                            "Seoul_Gangbuk-gu":str(test_predict_to_original[:,8]),
                            "Seoul_Dobong-gu":str(test_predict_to_original[:,9]),
                            "Seoul_Nowon-gu":str(test_predict_to_original[:,10]),
                            "Seoul_Eunpyeong-gu":str(test_predict_to_original[:,11]),
                            "Seoul_Seodaemun-gu":str(test_predict_to_original[:,12]),
                            "Seoul_Mapo-gu":str(test_predict_to_original[:,13]),
                            "Seoul_Yangcheon-gu":str(test_predict_to_original[:,14]),
                            "Seoul_Gangseo-gu":str(test_predict_to_original[:,15]),
                            "Seoul_Guro-gu":str(test_predict_to_original[:,16]),
                            "Seoul_Geumcheon-gu":str(test_predict_to_original[:,17]),
                            "Seoul_Yeongdeungpo-gu":str(test_predict_to_original[:,18]),
                            "Seoul_Dongjak-gu":str(test_predict_to_original[:,19]),
                            "Seoul_Gwanak-gu":str(test_predict_to_original[:,20]),
                            "Seoul_Seocho-gu":str(test_predict_to_original[:,21]),
                            "Seoul_Gangnam-gu":str(test_predict_to_original[:,22]),
                            "Seoul_Songpa-gu":str(test_predict_to_original[:,23]),
                            "Seoul_Gangdong-gu":str(test_predict_to_original[:,24])}

#Output to a file
with open("/Users/yeseo/Desktop/predict_"+str(2) +".json",'w',encoding = "utf-8") as make_file:
    json.dump(file_data,make_file,ensure_ascii =False,indent = "\t")

In [4]:
np.shape(test_predict_to_original)

(49, 25)